In [1]:
# Dependencies
import pandas as pd
import pymongo
import time
import datetime as dt

In [2]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.housing_db
collection = db.housing_summary
# df = pd.DataFrame(list(db.redfin.find().limit(100000)))
# df = pd.DataFrame(list(db.redfin.find()))

In [ ]:
sel_cols = ['period_begin',  
'period_end', 
'region_name' ,
'region_type',
'total_homes_sold',
'median_sale_price',
'median_sale_ppsf',
'median_new_listing_price',
'pending_sales',
'total_new_listings',
'price_drops',
'homes_delisted',
'inventory',
'total_active_listings',
'median_days_on_market'
]

In [ ]:
df_reduced = df[sel_cols]

In [ ]:
# df_reduced["period_begin"].to_datetime()
# 
begin_dates = pd.to_datetime(df_reduced["period_begin"], infer_datetime_format=True)
df_reduced["year"] = begin_dates.dt.strftime("%Y")
df_reduced["month"] = begin_dates.dt.strftime("%m")

df_reduced["year_month"] = begin_dates.dt.strftime("%Y-%m")

states = []
for region in df_reduced["region_name"]:
    reg_list = region.split(",")
    if len(reg_list) == 2:
        reg_state = reg_list[1][1:3]
        states.append(reg_list[1][1:3])
    else:
        states.append("")
df_reduced["state"] = states


In [ ]:
df = df_reduced
df.fillna('', inplace=True)
df = df[(df.year == '2020')& (df.state == 'NY')]
df


In [ ]:
df[['price_drops','inventory', 'median_days_on_market','median_sale_price' ]]= df[['price_drops','inventory', 'median_days_on_market','median_sale_price' ]].apply(pd.to_numeric)
df.dtypes

In [ ]:
df= df.groupby(['month','region_name']).agg({'inventory':'mean',
                         'median_days_on_market':'mean',
                         'median_sale_price':'mean',
                         'price_drops':'mean',
                        })
df

In [19]:
df.reset_index(inplace = True)
data_dict = df.to_dict("records")

In [20]:
 # Insert dictionary into MongoDB as a document
    collection.insert_many(data_dict)

In [4]:
df = pd.DataFrame(list(db.housing_summary.find()))
df

,_id,month,region_name,inventory,median_days_on_market,median_sale_price,price_drops
0,60952012d1bec65f2c9384ba,01,"Albany County, NY",7941.0,58.658333,218320.825000,361.0
1,60952012d1bec65f2c9384bb,01,"Albany, NY metro area",30142.0,61.941667,206972.016667,1167.0
2,60952012d1bec65f2c9384bc,01,"Allegany County, NY",1533.0,95.091667,65026.925000,34.0
3,60952012d1bec65f2c9384bd,01,"Amsterdam, NY metro area",1881.0,85.541667,106653.366667,63.0
4,60952012d1bec65f2c9384be,01,"Auburn, NY metro area",2553.0,74.375000,115040.266667,87.0
...,...,...,...,...,...,...,...
619,60952012d1bec65f2c938725,12,"Warren County, NY",2929.0,61.991667,232587.141667,61.0
620,60952012d1bec65f2c938726,12,"Washington County, NY",2285.0,81.308333,167634.916667,49.0
621,60952012d1bec65f2c938727,12,"Wayne County, NY",984.0,17.741667,155767.033333,31.0
622,60952012d1bec65f2c938728,12,"Westchester County, NY",30541.0,41.150000,601293.875000,660.0


In [7]:
df['median_days_on_market', 'median_sale_price'] = df['median_days_on_market','median_sale_price'].round(2)

SyntaxError: invalid syntax (<ipython-input-7-4c47d03ed132>, line 1)